# Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---


### Step 1: Compute the camera calibration matrix and distortion coefficients given a set of chessboard images

In [ ]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
%matplotlib qt5
%matplotlib inline

def camera_calibra(objpoints, imgpoints, img_2d):
    return cv2.calibrateCamera(objpoints, imgpoints, img_2d, None, None)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('./camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
    
    # If found, add object points, image points
    if ret == True:
        print(fname)
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
        
        ret, camera_mtx, dist_coeff, rvecs, tvecs = camera_calibra(objpoints, imgpoints, gray.shape[0:2])
                
        undist = cv2.undistort(img, camera_mtx, dist_coeff, None, camera_mtx)
        
        plt.figure()
        plt.imshow(undist)
plt.savefig('./output_images/'+'chessboard_undistort', bbox_inches='tight')
print('Final camera matrix = {}'.format(camera_mtx))
print('Final distortion coefficients = {}'.format(dist_coeff))
        #cv2.imshow('img',dst)
        #cv2.waitKey(500)
#cv2.destroyAllWindows()

### Step 2: Apply a distortion correction to raw images.

In [ ]:
img = cv2.imread('./test_images/test4.jpg')
print(img.shape)
undist = cv2.undistort(img, camera_mtx, dist_coeff, None, camera_mtx)
print(undist.shape)

f, ax = plt.subplots(1, 2, figsize=(32,32))
f.tight_layout()

ax[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
ax[0].set_title('Original Image', fontsize=50)
ax[1].imshow(cv2.cvtColor(undist, cv2.COLOR_BGR2RGB))
ax[1].set_title('Distortion Correction', fontsize=50)
plt.savefig('./output_images/'+'undistorted', bbox_inches='tight')

### Step 3: Use color transforms, gradients, etc., to create a thresholded binary image

In [ ]:
def hsv_select(img, thresh=(0, 255)):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) # H in [0,180], S and V in [0,255]
    s_channel = hsv[:,:,1]
    v_channel = hsv[:,:,2]
    bin_s = np.zeros_like(s_channel)
    bin_v = np.zeros_like(v_channel)
    bin_s[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
    bin_v[(v_channel > thresh[0]) & (v_channel <= thresh[1])] = 1
    binary_output = np.zeros_like(hsv[:,:,0])
    binary_output[(bin_s==1) & (bin_v==1)] = 1
    return binary_output

def hls_select(img, thresh=(0, 255)):
    hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS) # H in [0,180], L and S in [0,255]
    s_channel = hls[:,:,2]
    l_channel = hls[:,:,1]
    bin_s = np.zeros_like(s_channel)
    bin_s[(s_channel > thresh[0]) & (s_channel <= thresh[1])] = 1
    bin_l = np.zeros_like(l_channel)
    bin_l[(l_channel > thresh[0]) & (l_channel <= thresh[1])] = 1
    
    binary_output = np.zeros_like(hls[:,:,0])
    binary_output[(bin_s==1) & (bin_l==1)] = 1
    return binary_output

def abs_sobel_thresh(img, orient='x', thresh_min=0, thresh_max=255):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1

    # Return the result
    return binary_output

def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1

    # Return the binary image
    return binary_output

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output

test_img = cv2.imread('./test_images/test4.jpg')
print('Image shape is {}'.format(test_img.shape))
gradx = abs_sobel_thresh(test_img, orient='x', thresh_min=70, thresh_max=255)
grady = abs_sobel_thresh(test_img, orient='y', thresh_min=50, thresh_max=255)
mag_binary = mag_thresh(test_img, sobel_kernel=3, mag_thresh=(90, 128))
dir_binary = dir_threshold(test_img, sobel_kernel=15, thresh=(0.9, 1.1))
combined = np.zeros_like(dir_binary)
combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
hls_binary = hls_select(test_img, thresh=(90, 255))
hsv_binary = hsv_select(test_img, thresh=(90, 255))

# Plot the result
f, ax = plt.subplots(4, 2, figsize=(32, 32))
f.tight_layout()

ax[0,0].imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))
ax[0,0].set_title('Original Image', fontsize=50)

ax[0,1].imshow(gradx, cmap='gray')
ax[0,1].set_title('Thresholded Gradient X', fontsize=50)

ax[1,0].imshow(grady, cmap='gray')
ax[1,0].set_title('Thresholded Gradient Y', fontsize=50)

ax[1,1].imshow(mag_binary, cmap='gray')
ax[1,1].set_title('Thresholded Magnitude', fontsize=50)

ax[2,0].imshow(dir_binary, cmap='gray')
ax[2,0].set_title('Thresholded Grad. Dir.', fontsize=50)

ax[2,1].imshow(combined, cmap='gray')
ax[2,1].set_title('Combined', fontsize=50)
print(hls_binary.shape)
ax[3,0].imshow(hls_binary, cmap='gray')
ax[3,0].set_title('HLS', fontsize=50)

ax[3,1].imshow(hsv_binary, cmap='gray')
ax[3,1].set_title('HSV', fontsize=50)

#ax[-1, -1].axis('off')
plt.subplots_adjust(left=0., top=0.9, bottom=0., hspace=0.2)
plt.savefig('./output_images/'+'binary_threshold', bbox_inches='tight')

### Threshold Binary Pipeline

In [ ]:
def binary_pipeline(image, s_thresh=(170, 255), sx_thresh=(20, 100)):
    img = np.copy(image)
    
    gradx = abs_sobel_thresh(img, orient='x', thresh_min=70, thresh_max=128)
    grady = abs_sobel_thresh(img, orient='y', thresh_min=70, thresh_max=128)
    mag_binary = mag_thresh(img, sobel_kernel=3, mag_thresh=(90, 128))
    dir_binary = dir_threshold(img, sobel_kernel=15, thresh=(0.9, 1.1))
    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    hls_binary = hls_select(img, thresh=(85, 255))
    hsv_binary = hsv_select(img, thresh=(85, 255))

    binary = combined + hls_binary + hsv_binary
    #print(binary.shape)
    
    binary_output =  np.zeros_like(binary)
    binary_output[binary >= 1] = 1
    
    return binary_output

bin_undist = binary_pipeline(undist)
print(bin_undist.shape)
f, ax = plt.subplots(1, 2, figsize=(32, 32))
f.tight_layout()

ax[0].imshow(cv2.cvtColor(undist, cv2.COLOR_BGR2RGB))
ax[0].set_title('RGB', fontsize=50)
ax[1].imshow(bin_undist, cmap='gray')
ax[1].set_title('Threshold Binary', fontsize=50)
plt.savefig('./output_images/'+'binary_pipeline', bbox_inches='tight')

images = glob.glob('./test_images/*.jpg')

for fname in images:
    i = cv2.imread(fname)
    f, ax = plt.subplots(1, 2, figsize=(32, 32))
    ax[0].imshow(cv2.cvtColor(i, cv2.COLOR_BGR2RGB))
    ax[1].imshow(binary_pipeline(i), cmap='gray')


### Step 4: Apply a perspective transform to rectify binary image ("birds-eye view")

In [ ]:
def corners_unwarp(img, nx, ny, mtx, dist):
    # Use the OpenCV undistort() function to remove distortion
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    # Convert undistorted image to grayscale
    gray = cv2.cvtColor(undist, cv2.COLOR_BGR2GRAY)
    # Search for corners in the grayscaled image
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

    if ret == True:
        # If we found corners, draw them! (just for fun)
        cv2.drawChessboardCorners(undist, (nx, ny), corners, ret)
        # Choose offset from image corners to plot detected corners
        # This should be chosen to present the result at the proper aspect ratio
        # My choice of 100 pixels is not exact, but close enough for our purpose here
        offset = 100 # offset for dst points
        # Grab the image shape
        img_size = (gray.shape[1], gray.shape[0])

        # For source points I'm grabbing the outer four detected corners
        src = np.float32([corners[0], corners[nx-1], corners[-1], corners[-nx]])
        # For destination points, I'm arbitrarily choosing some points to be
        # a nice fit for displaying our warped result 
        # again, not exact, but close enough for our purposes
        dst = np.float32([[offset, offset], [img_size[0]-offset, offset], 
                                     [img_size[0]-offset, img_size[1]-offset], 
                                     [offset, img_size[1]-offset]])
        # Given src and dst points, calculate the perspective transform matrix
        M = cv2.getPerspectiveTransform(src, dst)
        # Warp the image using OpenCV warpPerspective()
        warped = cv2.warpPerspective(undist, M, img_size)

    # Return the resulting image and matrix
    return warped, M

# Warped example image
test_img = cv2.imread('./camera_cal/calibration3.jpg')
warp_test_img, M = corners_unwarp(test_img, 9, 6, camera_mtx, dist_coeff)
print(warp_test_img.shape)
f, ax = plt.subplots(1, 2, figsize=(32, 32))
f.tight_layout()
ax[0].imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))
ax[0].set_title('Original', fontsize=50)
ax[1].imshow(warp_test_img, cmap='gray')
ax[1].set_title('Undistorted and Warped', fontsize=50)
plt.savefig('./output_images/'+'example_undist_warp', bbox_inches='tight')

top_left_src = [600, 445]#[610,445]
top_right_src = [710, 445]#[725,445]
bottom_left_src = [240, 719]#[225,719]
bottom_right_src = [1120, 719]#[1110,719]

tmp_undist = np.copy(undist)
cv2.line(tmp_undist, (top_left_src[0], top_left_src[1]), (bottom_left_src[0], bottom_left_src[1]),[255, 0, 0], thickness=3)
cv2.line(tmp_undist, (top_right_src[0], top_right_src[1]), (bottom_right_src[0], bottom_right_src[1]),[255, 0, 0], thickness=3)

top_left_dst = [320, 0]#[250, 0]
top_right_dst = [960, 0]#[1030, 0]
bottom_left_dst = [320, 719]#[250,719] 
bottom_right_dst = [960, 719]#[1030, 719]

src = np.float32([bottom_left_src,bottom_right_src,top_right_src,top_left_src])
dst = np.float32([bottom_left_dst,bottom_right_dst,top_right_dst,top_left_dst])
M = cv2.getPerspectiveTransform(src, dst)
M_inv = cv2.getPerspectiveTransform(dst, src)
img_size = (undist.shape[1], undist.shape[0])

warp_bin_undist = cv2.warpPerspective(bin_undist, M, img_size)
warp_bin_undist[:,0:warp_bin_undist.shape[1]//4-200] = 0
warp_bin_undist[:,warp_bin_undist.shape[1]*3//4+200:] = 0
warp_bin_undist[:,warp_bin_undist.shape[1]//2-100:warp_bin_undist.shape[1]//2+100] = 0

warp_undist = cv2.warpPerspective(undist, M, img_size)

f, ax = plt.subplots(2, 2, figsize=(32, 24))
f.tight_layout()

ax[0,0].imshow(cv2.cvtColor(tmp_undist, cv2.COLOR_BGR2RGB))
ax[0,0].set_title('Undistorted', fontsize=50)
ax[0,1].imshow(cv2.cvtColor(warp_undist, cv2.COLOR_BGR2RGB))
ax[0,1].set_title('Undistorted and Warped', fontsize=50)
ax[1,0].imshow(bin_undist, cmap='gray')
ax[1,0].set_title('Binary Undistorted', fontsize=50)
ax[1,1].imshow(warp_bin_undist, cmap='gray')
ax[1,1].set_title('Binary Undistorted and Warped', fontsize=50)

plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0., hspace=0.2, wspace=0.2)
plt.savefig('./output_images/'+'undistorted_warped', bbox_inches='tight')

### Step 5: Detect lane pixels and fit to find the lane boundary.

#### Sliding Window Search

In [ ]:
# Read in a thresholded image
warped = np.copy(warp_bin_undist)

# window settings
window_width = 80 # 
window_height = 60 # 80, Break image into 9 vertical layers since image height is 720
margin = 100 # 100, How much to slide left and right for searching

def window_mask(width, height, img_ref, center,level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
    return output

def find_window_centroids(image, window_width, window_height, margin):
    
    window_centroids = [] # Store the (left,right) window centroid positions per level
    window = np.ones(window_width) # Create our window template that we will use for convolutions
    
    # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
    # and then np.convolve the vertical image slice with the window template 
    
    # Sum quarter bottom of image to get slice, could use a different ratio
    l_sum = np.sum(warped[int(3*warped.shape[0]/4):,:int(warped.shape[1]/2)], axis=0)
    l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
    r_sum = np.sum(warped[int(3*warped.shape[0]/4):,int(warped.shape[1]/2):], axis=0)
    r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(warped.shape[1]/2)
    #print('left center is {}, right center is {}'.format(l_center, r_center))
    # Add what we found for the first layer
    window_centroids.append((l_center,r_center))
    
    # Go through each layer looking for max pixel locations
    for level in range(1,(int)(warped.shape[0]/window_height)):
        # convolve the window into the vertical slice of the image
        image_layer = np.sum(warped[int(warped.shape[0]-(level+1)*window_height):int(warped.shape[0]-level*window_height),:], axis=0)
        conv_signal = np.convolve(window, image_layer)
        # Find the best left centroid by using past left center as a reference
        # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
        offset = window_width/2
        l_min_index = int(max(l_center+offset-margin,0))
        l_max_index = int(min(l_center+offset+margin,warped.shape[1]))
        l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
        # Find the best right centroid by using past right center as a reference
        r_min_index = int(max(r_center+offset-margin,0))
        r_max_index = int(min(r_center+offset+margin,warped.shape[1]))
        r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
        #print(l_center, r_center)
        # Add what we found for that layer
        window_centroids.append((l_center,r_center))

    return window_centroids

def find_lanelines(image, window_centroids, window_width, window_height):
    warped = np.copy(image)
    # If we found any window centers
    if len(window_centroids) > 0:
    
        # Points used to draw all the left and right windows
        l_points = np.zeros_like(warped)
        r_points = np.zeros_like(warped)
    
        # Go through each level and draw the windows
        for level in range(0,len(window_centroids)):
            # Window_mask is a function to draw window areas
            l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
            r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
            # Add graphic points from window mask here to total pixels found
            l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
            r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255
    
        # Draw the results
        template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
        zero_channel = np.zeros_like(template) # create a zero color channel
        template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
        warpage = np.array(cv2.merge((warped*255,warped*255,warped*255)),np.uint8) # making the original road pixels 3 color channels
        output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results
        return warpage, template
    # If no window centers found, just display orginal road image
    else:
        warpage = np.array(cv2.merge((warped*255,warped*255,warped*255)),np.uint8)
        output = np.array(cv2.merge((warped*255,warped*255,warped*255)),np.uint8)
        return warpage
    
window_centroids = find_window_centroids(warped, window_width, window_height, margin)
output = find_lanelines(warped, window_centroids, window_width, window_height)
if len(output) == 2:
    warpage = output[0]
    template = output[1]
    leftx = np.copy(np.where(template, warpage, 0))[:,:,1]
    rightx = np.copy(np.where(template, warpage, 0))[:,:,1]
    leftx[:,leftx.shape[1]//2:] = 0
    rightx[:,0:rightx.shape[1]//2] = 0
elif len(output) == 1:
    print('length == 1')
else:
    exit(1)

print('warpage\'s shape', warpage.shape, 'template\'s shape', template.shape)
# Display the final results
plt.imshow(cv2.addWeighted(warpage, 1, template, 0.5, 0.0))
plt.title('window fitting results')
plt.savefig('./output_images/'+'window_fitting', bbox_inches='tight')

f, ax = plt.subplots(1, 2, figsize=(32, 24))
f.tight_layout()
ax[0].imshow(leftx, cmap='gray')
ax[0].set_title('Left Lane Line', fontsize=50)
ax[1].imshow(rightx, cmap='gray')
ax[1].set_title('Right Lane Line', fontsize=50)


### Step 6: Determine the curvature of the lane and vehicle position with respect to center.

In [ ]:
def find_lane_curve(ploty, leftx, rightx):
    y_axis_left = np.where(leftx != 0)[0]
    x_leftx = np.where(leftx != 0)[1]

    y_axis_right = np.where(rightx != 0)[0]
    x_rightx = np.where(rightx != 0)[1]
    
    if x_leftx.shape[0] == 0 or x_rightx.shape[0] == 0:
        return None, None, None, None, None, None, None, None
    #print(y_axis_left.shape, x_leftx.shape)
    left_fit = np.polyfit(y_axis_left, x_leftx, 2)
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fit = np.polyfit(y_axis_right, x_rightx, 2)
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    #left_fit = np.polyfit(ploty, leftx, 2)
    #right_fit = np.polyfit(ploty, rightx, 2)
    return left_fit, left_fitx, right_fit, right_fitx, x_leftx, x_rightx, y_axis_left, y_axis_right

In [ ]:
# Fit a second order polynomial to pixel positions in each fake lane line
ploty = np.linspace(0, 719, num=720)# to cover same y-range as image

left_fit, left_fitx, right_fit, right_fitx, leftx, rightx, y_axis_left, y_axis_right = find_lane_curve(ploty, leftx, rightx)

plt.imshow(np.copy(np.where(template, warpage, 0))[:,:,1], cmap='gray')

plt.xlim(0, 1280)
plt.ylim(0, 720)
plt.plot(left_fitx, ploty, color='green', linewidth=3)
plt.plot(right_fitx, ploty, color='green', linewidth=3)
plt.gca().invert_yaxis() # to visualize as we do the images
plt.savefig('./output_images/'+'find_curve', bbox_inches='tight')

In [ ]:
#### Calculated the radius of curvature of the lane

In [ ]:
def get_curvature(ploty, left_fit, right_fit, leftx, rightx, y_axis_left, y_axis_right):
    # Define y-value where we want radius of curvature
    # I'll choose the maximum y-value, corresponding to the bottom of the image
    y_eval = np.max(ploty)
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    #print(left_curverad, right_curverad)
    # Example values: 1926.74 1908.48

    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(y_axis_left*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(y_axis_right*ym_per_pix, rightx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    # Now our radius of curvature is in meters
    #print(left_curverad, 'm', right_curverad, 'm')
    # Example values: 632.1 m    626.2 m
    
    return left_curverad, right_curverad

In [ ]:
left_curverad, right_curverad = get_curvature(ploty, left_fit, right_fit, leftx, rightx, y_axis_left, y_axis_right)
print(left_curverad, 'm', right_curverad, 'm')


### Step 7: Warp the detected lane boundaries back onto the original image

In [ ]:
def warp_lane_back(warped, undist, left_fitx, right_fitx):
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, M_inv, (img.shape[1], img.shape[0])) 

    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)

    return result

result = warp_lane_back(warped, undist, left_fitx, right_fitx)

plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))

plt.savefig('./output_images/'+'final', bbox_inches='tight')


### Step 8: Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

In [ ]:
print('Left lane curvature is {}m, right lane curvature is {}m'.format(left_curverad, right_curverad))

'''midpoint = (left_fitx[719] + right_fitx[719])/2
offset = (midpoint - 640)
if offset > 0:
    print('Vehicle is {}m right of center'.format(offset))
elif offset < 0:
    print('Vehicle is {}m left of center'.format(offset))
else:
    print('Vehicle is at the center')'''

## Process Video

In [ ]:
# Import everything needed to edit/save/watch video clips
import imageio
#imageio.plugins.ffmpeg.download()
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
        
        self.last_left_fitx = None
        self.last_right_fitx = None
        self.last_left_curverad = None
        self.last_right_curverad = None
        
history_line = Line()
history_line.last_left_fitx = left_fitx
history_line.last_right_fitx = right_fitx
history_line.last_left_curverad = 500
history_line.last_right_curverad = 500

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) 
    # where lines are drawn on lanes.
    img = np.copy(image)
    undist = cv2.undistort(img, camera_mtx, dist_coeff, None, camera_mtx)
    bin_undist = binary_pipeline(undist)
    
    warp_bin_undist = cv2.warpPerspective(bin_undist, M, img_size)
    warp_bin_undist[:,0:warp_bin_undist.shape[1]//4-150] = 0
    warp_bin_undist[:,warp_bin_undist.shape[1]*3//4+150:] = 0
    warp_bin_undist[:,warp_bin_undist.shape[1]//2-100:warp_bin_undist.shape[1]//2+100] = 0

    window_centroids = find_window_centroids(warp_bin_undist, window_width, window_height, margin)
    
    output = find_lanelines(warp_bin_undist, window_centroids, window_width, window_height)
    
    if len(output) == 2:
        warpage = output[0]
        template = output[1]
        leftx = np.copy(np.where(template, warpage, 0))[:,:,1]
        rightx = np.copy(np.where(template, warpage, 0))[:,:,1]
        leftx[:,leftx.shape[1]//2:] = 0
        rightx[:,0:rightx.shape[1]//2] = 0
    elif len(output) == 1:
        return output
    else:
        exit(1)

    ploty = np.linspace(0, 719, num=720)# to cover same y-range as image
    left_fit, left_fitx, right_fit, right_fitx, leftx, rightx, y_axis_left, y_axis_right = find_lane_curve(ploty, leftx, rightx)
    
    if left_fitx is None and right_fitx is None:
        
        return undist

    left_curverad, right_curverad = get_curvature(ploty, left_fit, right_fit, leftx, rightx, y_axis_left, y_axis_right)
    #print(left_curverad, 'm', right_curverad, 'm')
    #if 100 < left_curverad < 1000 and 100 < right_curverad < 1000 and (abs(history_line.last_left_curverad-left_curverad) < 300) and (abs(history_line.last_right_curverad-right_curverad) < 300):
    if (abs(history_line.last_left_curverad-left_curverad) < 300) and (abs(history_line.last_right_curverad-right_curverad) < 150): # for normal line
        history_line.last_left_fitx = left_fitx
        history_line.last_right_fitx = right_fitx
        history_line.last_left_curverad = left_curverad
        history_line.last_right_curverad = right_curverad
        result = warp_lane_back(warp_bin_undist, undist, left_fitx, right_fitx)
    elif (300 < left_curverad < 2000) and (300 < right_curverad < 2000): # for normal line
        history_line.last_left_fitx = left_fitx
        history_line.last_right_fitx = right_fitx
        history_line.last_left_curverad = left_curverad
        history_line.last_right_curverad = right_curverad
        result = warp_lane_back(warp_bin_undist, undist, left_fitx, right_fitx)
    elif left_curverad > 5000 and right_curverad > 5000: # for straight line
        history_line.last_left_fitx = left_fitx
        history_line.last_right_fitx = right_fitx
        history_line.last_left_curverad = left_curverad
        history_line.last_right_curverad = right_curverad
        result = warp_lane_back(warp_bin_undist, undist, left_fitx, right_fitx)
    else:
        result = warp_lane_back(warp_bin_undist, undist, history_line.last_left_fitx, history_line.last_right_fitx)
        
    return result

In [ ]:
input_video = 'project_video.mp4'
output_video = 'output_videos/project_video.mp4'

## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip1 = VideoFileClip(input_video).subclip(5,7) # 20-25
clip1 = VideoFileClip(input_video)
out_clip1 = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time out_clip1.write_videofile(output_video, audio=False)
